In [88]:
import pandas as pd
pd.set_option('display.float_format', '{:.5f}'.format)
import numpy as np

In [89]:
ticker="tza"
df=pd.read_csv(f"data/raw/{ticker}_20250103_20250131.csv",engine="pyarrow", sep = ',')
df["Date-Time"] = df["Date-Time"].dt.tz_convert("America/New_York")
df.head(3)
df.describe()

,L1-BidPrice,L1-BidSize,L1-AskPrice,L1-AskSize,L2-BidPrice,L2-BidSize,L2-AskPrice,L2-AskSize,L3-BidPrice,L3-BidSize,...,L23-AskPrice,L23-AskSize,L24-BidPrice,L24-BidSize,L24-AskPrice,L24-AskSize,L25-BidPrice,L25-BidSize,L25-AskPrice,L25-AskSize
count,9947378.00000,9947378.00000,9947629.00000,9947629.00000,9946471.00000,9946471.00000,9946872.00000,9946872.00000,9945264.00000,9945264.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
mean,12.48407,31013.73044,12.49560,22733.27859,12.47399,31861.17940,12.50567,28275.88622,12.46389,31321.93379,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,0.73356,23752.96063,0.73378,18894.36265,0.73356,18777.01226,0.73382,17045.00919,0.73356,18664.97923,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,11.15000,1.00000,11.49000,1.00000,10.72000,4.00000,11.50000,26.00000,9.52000,4.00000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,11.92000,13900.00000,11.93000,8200.00000,11.91000,18200.00000,11.94000,16800.00000,11.90000,17700.00000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,12.26000,25600.00000,12.27000,18000.00000,12.25000,29000.00000,12.28000,25500.00000,12.24000,28700.00000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,12.99000,43800.00000,13.00000,32400.00000,12.98000,43800.00000,13.01000,38000.00000,12.97000,43600.00000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,14.28000,266400.00000,14.36000,265000.00000,14.27000,262600.00000,14.36000,264000.00000,14.26000,170900.00000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [90]:
pd.DataFrame(pd.to_datetime(df["Date-Time"]).dt.date.unique())

,0
0,2025-01-03
1,2025-01-06
2,2025-01-07
3,2025-01-08
4,2025-01-10
5,2025-01-13
6,2025-01-14
7,2025-01-15
8,2025-01-16
9,2025-01-17


In [91]:
df["Date"] = pd.to_datetime(df["Date-Time"]).dt.date
grouped = df.groupby("Date").size().reset_index(name="Count")


In [92]:
# Filter only regular market hours (09:30–16:00 Eastern)
df_market = df[
    (df["Date-Time"].dt.time >= pd.to_datetime("09:40:00").time()) &
    (df["Date-Time"].dt.time <= pd.to_datetime("15:50:00").time())
]

In [93]:
base_cols = ["#RIC", "Date-Time"]
#base_cols = ["#RIC"]
# Dynamically build list for L1–L10
levels = []
for i in range(1, 11):  # L1–L10 inclusive
    levels += [
        f"L{i}-BidPrice", f"L{i}-BidSize",
        f"L{i}-AskPrice", f"L{i}-AskSize"
    ]


In [94]:
df_market_l10 = df_market[base_cols + levels]

In [95]:
df_market_l10.head(10)
df_market_l10.describe()

,L1-BidPrice,L1-BidSize,L1-AskPrice,L1-AskSize,L2-BidPrice,L2-BidSize,L2-AskPrice,L2-AskSize,L3-BidPrice,L3-BidSize,...,L8-AskPrice,L8-AskSize,L9-BidPrice,L9-BidSize,L9-AskPrice,L9-AskSize,L10-BidPrice,L10-BidSize,L10-AskPrice,L10-AskSize
count,8734128.00000,8734128.00000,8734128.00000,8734128.00000,8734128.00000,8734128.00000,8734128.00000,8734128.00000,8734128.00000,8734128.00000,...,8734013.00000,8734013.00000,8734121.00000,8734121.00000,8733626.00000,8733626.00000,8734101.00000,8734101.00000,8733207.00000,8733207.00000
mean,12.49539,32763.45452,12.50667,23735.73235,12.48538,33281.73004,12.51668,29335.27542,12.47538,32886.08940,...,12.57960,17995.97994,12.41533,19270.94511,12.59095,16371.00562,12.40531,13460.53966,12.60552,12424.53259
std,0.73950,22875.43813,0.73969,18172.68068,0.73950,17039.69621,0.73969,15319.89583,0.73949,17144.22732,...,0.80821,9016.40025,0.73950,8632.10686,0.81499,7749.52933,0.73953,9100.12000,0.84516,7947.32594
min,11.46000,1.00000,11.49000,1.00000,11.45000,100.00000,11.50000,96.00000,11.38000,100.00000,...,11.56000,10.00000,8.55000,100.00000,11.57000,4.00000,8.55000,100.00000,11.58000,4.00000
25%,11.91000,16000.00000,11.92000,9600.00000,11.90000,20100.00000,11.93000,18100.00000,11.89000,19600.00000,...,12.00000,15500.00000,11.83000,17000.00000,12.01000,15300.00000,11.82000,4100.00000,12.02000,4000.00000
50%,12.31000,27900.00000,12.32000,19300.00000,12.30000,30800.00000,12.33000,26700.00000,12.29000,30500.00000,...,12.39000,17400.00000,12.23000,19800.00000,12.40000,16800.00000,12.22000,16300.00000,12.42000,15500.00000
75%,13.03000,45400.00000,13.04000,33500.00000,13.02000,44700.00000,13.05000,38600.00000,13.01000,44600.00000,...,13.11000,20600.00000,12.95000,22810.00000,13.12000,18600.00000,12.94000,20000.00000,13.13000,17200.00000
max,14.28000,218953.00000,14.29000,193700.00000,14.27000,218953.00000,14.30000,193700.00000,14.26000,100900.00000,...,65.00000,74500.00000,14.20000,72600.00000,65.00000,62100.00000,14.19000,67900.00000,65.00000,62100.00000


In [96]:
price_cols = df_market_l10.filter(regex=r'^L(?:10|[1-9])-(?:Bid|Ask)Price$').columns

# Keep only rows where ANY price column changed vs the previous row
changed = df_market_l10[price_cols].ne(df_market_l10[price_cols].shift()).any(axis=1)
df_filtered = df_market_l10[changed].copy()

In [97]:
print(len(df_filtered))
print(len(df_market_l10))

247876
8734128


In [98]:
df_market_l10=df_filtered

In [99]:
df_market_l10["ms_timestamp"] = df_market_l10["Date-Time"].dt.floor("ms")
df_market_l10 = (
   df_market_l10.drop_duplicates(subset=["#RIC", "ms_timestamp"], keep="last")
     .drop(columns="ms_timestamp")
)
df_market_l10=df_market_l10.dropna()

In [100]:
df_market_l10.head(10)
df_market_l10.describe()

,L1-BidPrice,L1-BidSize,L1-AskPrice,L1-AskSize,L2-BidPrice,L2-BidSize,L2-AskPrice,L2-AskSize,L3-BidPrice,L3-BidSize,...,L8-AskPrice,L8-AskSize,L9-BidPrice,L9-BidSize,L9-AskPrice,L9-AskSize,L10-BidPrice,L10-BidSize,L10-AskPrice,L10-AskSize
count,211291.00000,211291.00000,211291.00000,211291.00000,211291.00000,211291.00000,211291.00000,211291.00000,211291.00000,211291.00000,...,211291.00000,211291.00000,211291.00000,211291.00000,211291.00000,211291.00000,211291.00000,211291.00000,211291.00000,211291.00000
mean,12.58284,25599.10372,12.59754,22519.38309,12.57282,30401.88722,12.60758,27648.61953,12.56280,26413.30962,...,12.67098,15412.38978,12.50252,17245.20539,12.68546,14156.37304,12.49234,12179.47581,12.72893,10783.41734
std,0.75000,21748.11262,0.75028,19786.78598,0.75002,18538.50332,0.75030,17529.22295,0.75002,15963.99179,...,0.75047,9286.79230,0.75012,9012.33496,0.75206,8231.31207,0.75034,9043.83972,1.35647,7946.49382
min,11.46000,1.00000,11.49000,1.00000,11.45000,100.00000,11.50000,100.00000,11.44000,100.00000,...,11.56000,10.00000,9.36000,100.00000,11.57000,4.00000,8.55000,100.00000,11.58000,4.00000
25%,12.00000,8100.00000,12.01000,6800.00000,11.99000,17700.00000,12.02000,16400.00000,11.98000,15600.00000,...,12.09000,13500.00000,11.92000,15400.00000,12.10000,6400.00000,11.91000,2900.00000,12.12000,2300.00000
50%,12.39000,20100.00000,12.41000,17800.00000,12.38000,27002.00000,12.42000,24100.00000,12.37000,23700.00000,...,12.48000,16500.00000,12.31000,18500.00000,12.49000,16200.00000,12.30000,15300.00000,12.52000,14200.00000
75%,13.21000,36800.00000,13.22000,32600.00000,13.20000,40132.00000,13.23000,36300.00000,13.19000,36200.00000,...,13.29000,18500.00000,13.13000,21700.00000,13.31000,17600.00000,13.12000,18800.00000,13.32000,16500.00000
max,14.28000,158999.00000,14.29000,155900.00000,14.27000,218953.00000,14.30000,193700.00000,14.26000,100900.00000,...,16.18000,65500.00000,14.20000,72600.00000,17.13000,62100.00000,14.19000,67900.00000,65.00000,62100.00000


In [101]:
#df_market_l10.drop("mid_price",axis=1, inplace=True)
#df_market_l10.drop("abs.spread",axis=1, inplace=True)
#df_market_l10.drop("rel.spread",axis=1, inplace=True)

In [102]:
df_market_l10.insert(
            2, "mid_price", (df_market_l10["L1-BidPrice"] + df_market_l10["L1-AskPrice"]) / 2
        )  # Add the mid-price column to the orderbook dataframe.
df_market_l10.insert(
            2, "abs.spread", (df_market_l10["L1-AskPrice"] - df_market_l10["L1-BidPrice"])
        )
df_market_l10.insert(
            2, "rel.spread", (df_market_l10["L1-AskPrice"] - df_market_l10["L1-BidPrice"])/df_market_l10["mid_price"]
        )

In [103]:
df_market_l10.head(10)

,#RIC,Date-Time,rel.spread,abs.spread,mid_price,L1-BidPrice,L1-BidSize,L1-AskPrice,L1-AskSize,L2-BidPrice,...,L8-AskPrice,L8-AskSize,L9-BidPrice,L9-BidSize,L9-AskPrice,L9-AskSize,L10-BidPrice,L10-BidSize,L10-AskPrice,L10-AskSize
32461,TZA,2025-01-03 09:40:00.030944719-05:00,0.00158,0.02000,12.63000,12.62000,20400.00000,12.64000,23900.00000,12.61000,...,12.71000,21400.00000,12.54000,17100.00000,12.72000,15700.00000,12.53000,900.00000,12.73000,15700.00000
32465,TZA,2025-01-03 09:40:00.040908580-05:00,0.00079,0.01000,12.62500,12.62000,24200.00000,12.63000,300.00000,12.61000,...,12.70000,15700.00000,12.54000,17100.00000,12.71000,21400.00000,12.53000,2600.00000,12.72000,15700.00000
32490,TZA,2025-01-03 09:40:00.070433515-05:00,0.00158,0.02000,12.63000,12.62000,25000.00000,12.64000,18200.00000,12.61000,...,12.71000,21400.00000,12.54000,17100.00000,12.72000,15700.00000,12.53000,2600.00000,12.73000,15700.00000
32507,TZA,2025-01-03 09:40:00.250962344-05:00,0.00079,0.01000,12.63500,12.63000,5400.00000,12.64000,23700.00000,12.62000,...,12.71000,21400.00000,12.55000,18000.00000,12.72000,15700.00000,12.54000,17100.00000,12.73000,15700.00000
32517,TZA,2025-01-03 09:40:00.373402083-05:00,0.00158,0.02000,12.63000,12.62000,20400.00000,12.64000,23700.00000,12.61000,...,12.71000,21400.00000,12.54000,17100.00000,12.72000,15700.00000,12.53000,900.00000,12.73000,15700.00000
32526,TZA,2025-01-03 09:40:00.640232807-05:00,0.00079,0.01000,12.62500,12.62000,24900.00000,12.63000,5000.00000,12.61000,...,12.70000,15700.00000,12.54000,17100.00000,12.71000,21400.00000,12.53000,900.00000,12.72000,15700.00000
32585,TZA,2025-01-03 09:40:01.385915172-05:00,0.00158,0.02000,12.63000,12.62000,29600.00000,12.64000,18700.00000,12.61000,...,12.71000,21400.00000,12.54000,17100.00000,12.72000,15700.00000,12.53000,900.00000,12.73000,15700.00000
32595,TZA,2025-01-03 09:40:01.685655716-05:00,0.00079,0.01000,12.62500,12.62000,25100.00000,12.63000,1000.00000,12.61000,...,12.70000,15700.00000,12.54000,17100.00000,12.71000,21400.00000,12.53000,900.00000,12.72000,15700.00000
32623,TZA,2025-01-03 09:40:01.920566198-05:00,0.00158,0.02000,12.63000,12.62000,25400.00000,12.64000,18900.00000,12.61000,...,12.71000,21400.00000,12.54000,17100.00000,12.72000,15700.00000,12.53000,900.00000,12.73000,15700.00000
32626,TZA,2025-01-03 09:40:02.120697551-05:00,0.00079,0.01000,12.62500,12.62000,25100.00000,12.63000,4000.00000,12.61000,...,12.70000,15700.00000,12.54000,17100.00000,12.71000,21400.00000,12.53000,900.00000,12.72000,15700.00000


In [104]:
print(sum(round(df_market_l10["abs.spread"],2)==float(0.01)))
tick_size=sum(round(df_market_l10["abs.spread"],2)==float(0.01))/df_market_l10.size
print(tick_size)

119912
0.012611569404807167


In [105]:
df_market_l10.describe()

,rel.spread,abs.spread,mid_price,L1-BidPrice,L1-BidSize,L1-AskPrice,L1-AskSize,L2-BidPrice,L2-BidSize,L2-AskPrice,...,L8-AskPrice,L8-AskSize,L9-BidPrice,L9-BidSize,L9-AskPrice,L9-AskSize,L10-BidPrice,L10-BidSize,L10-AskPrice,L10-AskSize
count,211291.00000,211291.00000,211291.00000,211291.00000,211291.00000,211291.00000,211291.00000,211291.00000,211291.00000,211291.00000,...,211291.00000,211291.00000,211291.00000,211291.00000,211291.00000,211291.00000,211291.00000,211291.00000,211291.00000,211291.00000
mean,0.00117,0.01470,12.59019,12.58284,25599.10372,12.59754,22519.38309,12.57282,30401.88722,12.60758,...,12.67098,15412.38978,12.50252,17245.20539,12.68546,14156.37304,12.49234,12179.47581,12.72893,10783.41734
std,0.00079,0.00999,0.75012,0.75000,21748.11262,0.75028,19786.78598,0.75002,18538.50332,0.75030,...,0.75047,9286.79230,0.75012,9012.33496,0.75206,8231.31207,0.75034,9043.83972,1.35647,7946.49382
min,0.00070,0.01000,11.48500,11.46000,1.00000,11.49000,1.00000,11.45000,100.00000,11.50000,...,11.56000,10.00000,9.36000,100.00000,11.57000,4.00000,8.55000,100.00000,11.58000,4.00000
25%,0.00080,0.01000,12.00500,12.00000,8100.00000,12.01000,6800.00000,11.99000,17700.00000,12.02000,...,12.09000,13500.00000,11.92000,15400.00000,12.10000,6400.00000,11.91000,2900.00000,12.12000,2300.00000
50%,0.00085,0.01000,12.40000,12.39000,20100.00000,12.41000,17800.00000,12.38000,27002.00000,12.42000,...,12.48000,16500.00000,12.31000,18500.00000,12.49000,16200.00000,12.30000,15300.00000,12.52000,14200.00000
75%,0.00159,0.02000,13.21500,13.21000,36800.00000,13.22000,32600.00000,13.20000,40132.00000,13.23000,...,13.29000,18500.00000,13.13000,21700.00000,13.31000,17600.00000,13.12000,18800.00000,13.32000,16500.00000
max,0.03262,0.38000,14.28500,14.28000,158999.00000,14.29000,155900.00000,14.27000,218953.00000,14.30000,...,16.18000,65500.00000,14.20000,72600.00000,17.13000,62100.00000,14.19000,67900.00000,65.00000,62100.00000


In [106]:
df_market_l10

,#RIC,Date-Time,rel.spread,abs.spread,mid_price,L1-BidPrice,L1-BidSize,L1-AskPrice,L1-AskSize,L2-BidPrice,...,L8-AskPrice,L8-AskSize,L9-BidPrice,L9-BidSize,L9-AskPrice,L9-AskSize,L10-BidPrice,L10-BidSize,L10-AskPrice,L10-AskSize
32461,TZA,2025-01-03 09:40:00.030944719-05:00,0.00158,0.02000,12.63000,12.62000,20400.00000,12.64000,23900.00000,12.61000,...,12.71000,21400.00000,12.54000,17100.00000,12.72000,15700.00000,12.53000,900.00000,12.73000,15700.00000
32465,TZA,2025-01-03 09:40:00.040908580-05:00,0.00079,0.01000,12.62500,12.62000,24200.00000,12.63000,300.00000,12.61000,...,12.70000,15700.00000,12.54000,17100.00000,12.71000,21400.00000,12.53000,2600.00000,12.72000,15700.00000
32490,TZA,2025-01-03 09:40:00.070433515-05:00,0.00158,0.02000,12.63000,12.62000,25000.00000,12.64000,18200.00000,12.61000,...,12.71000,21400.00000,12.54000,17100.00000,12.72000,15700.00000,12.53000,2600.00000,12.73000,15700.00000
32507,TZA,2025-01-03 09:40:00.250962344-05:00,0.00079,0.01000,12.63500,12.63000,5400.00000,12.64000,23700.00000,12.62000,...,12.71000,21400.00000,12.55000,18000.00000,12.72000,15700.00000,12.54000,17100.00000,12.73000,15700.00000
32517,TZA,2025-01-03 09:40:00.373402083-05:00,0.00158,0.02000,12.63000,12.62000,20400.00000,12.64000,23700.00000,12.61000,...,12.71000,21400.00000,12.54000,17100.00000,12.72000,15700.00000,12.53000,900.00000,12.73000,15700.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9932602,TZA,2025-01-31 15:49:34.820226768-05:00,0.00083,0.01000,11.99500,11.99000,8000.00000,12.00000,17300.00000,11.98000,...,12.07000,1300.00000,11.91000,4500.00000,12.08000,5100.00000,11.90000,300.00000,12.09000,1300.00000
9932711,TZA,2025-01-31 15:49:46.685043080-05:00,0.00167,0.02000,11.99000,11.98000,4700.00000,12.00000,12800.00000,11.97000,...,12.07000,1300.00000,11.90000,300.00000,12.08000,5100.00000,11.89000,400.00000,12.09000,1300.00000
9932717,TZA,2025-01-31 15:49:46.848207774-05:00,0.00083,0.01000,11.99500,11.99000,3100.00000,12.00000,12200.00000,11.98000,...,12.07000,1300.00000,11.91000,4500.00000,12.08000,5100.00000,11.90000,300.00000,12.09000,1300.00000
9932727,TZA,2025-01-31 15:49:47.335360471-05:00,0.00167,0.02000,11.99000,11.98000,4400.00000,12.00000,14200.00000,11.97000,...,12.07000,1300.00000,11.90000,300.00000,12.08000,5100.00000,11.89000,400.00000,12.09000,1300.00000


In [107]:
df_market_l10[~df_market_l10.isna().any(axis=1)].to_csv(f"data/{ticker}_cleaned_jan2025.csv", index=False)

In [108]:
df_market_l10.describe()

,rel.spread,abs.spread,mid_price,L1-BidPrice,L1-BidSize,L1-AskPrice,L1-AskSize,L2-BidPrice,L2-BidSize,L2-AskPrice,...,L8-AskPrice,L8-AskSize,L9-BidPrice,L9-BidSize,L9-AskPrice,L9-AskSize,L10-BidPrice,L10-BidSize,L10-AskPrice,L10-AskSize
count,211291.00000,211291.00000,211291.00000,211291.00000,211291.00000,211291.00000,211291.00000,211291.00000,211291.00000,211291.00000,...,211291.00000,211291.00000,211291.00000,211291.00000,211291.00000,211291.00000,211291.00000,211291.00000,211291.00000,211291.00000
mean,0.00117,0.01470,12.59019,12.58284,25599.10372,12.59754,22519.38309,12.57282,30401.88722,12.60758,...,12.67098,15412.38978,12.50252,17245.20539,12.68546,14156.37304,12.49234,12179.47581,12.72893,10783.41734
std,0.00079,0.00999,0.75012,0.75000,21748.11262,0.75028,19786.78598,0.75002,18538.50332,0.75030,...,0.75047,9286.79230,0.75012,9012.33496,0.75206,8231.31207,0.75034,9043.83972,1.35647,7946.49382
min,0.00070,0.01000,11.48500,11.46000,1.00000,11.49000,1.00000,11.45000,100.00000,11.50000,...,11.56000,10.00000,9.36000,100.00000,11.57000,4.00000,8.55000,100.00000,11.58000,4.00000
25%,0.00080,0.01000,12.00500,12.00000,8100.00000,12.01000,6800.00000,11.99000,17700.00000,12.02000,...,12.09000,13500.00000,11.92000,15400.00000,12.10000,6400.00000,11.91000,2900.00000,12.12000,2300.00000
50%,0.00085,0.01000,12.40000,12.39000,20100.00000,12.41000,17800.00000,12.38000,27002.00000,12.42000,...,12.48000,16500.00000,12.31000,18500.00000,12.49000,16200.00000,12.30000,15300.00000,12.52000,14200.00000
75%,0.00159,0.02000,13.21500,13.21000,36800.00000,13.22000,32600.00000,13.20000,40132.00000,13.23000,...,13.29000,18500.00000,13.13000,21700.00000,13.31000,17600.00000,13.12000,18800.00000,13.32000,16500.00000
max,0.03262,0.38000,14.28500,14.28000,158999.00000,14.29000,155900.00000,14.27000,218953.00000,14.30000,...,16.18000,65500.00000,14.20000,72600.00000,17.13000,62100.00000,14.19000,67900.00000,65.00000,62100.00000
